## Model-Ready Census and SVI data

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

from us import states

from COS513 import load_all_years_of_noaa_storm_data
from COS513 import load_census_data
from COS513 import load_county_zone_mapping
from COS513 import decode_damage_property
from COS513 import download_census_data_for_all_states

In [21]:
noaa_data = load_all_years_of_noaa_storm_data()
# census_data = load_census_data()
county_zone_mapping = load_county_zone_mapping()

Loading NOAA storm data:   0%|          | 0/75 [00:00<?, ?it/s]

In [3]:
# load svi data
df_list_svi = []
data_years = [2000, 2010, 2014, 2016, 2018, 2020, 2022]
svi_path = os.path.join('data/SVI_County_Data/')

for yr in data_years:
    # read data
    print(yr)
    svi_path = os.path.join('data/SVI_County_Data/')
    filename = os.path.join(svi_path, 'SVI_' + str(yr) + '_US_county.csv')
    df = pd.read_csv(filename)

    # clean data
    df = df.replace(-999., np.nan) # nan values are labelled -999
    if yr == 2000:
        df = df.rename(columns={'USTP': 'RPL_THEMES', 'STATE_NAME': 'STATE', 'STCOFIPS': 'FIPS'})
    if yr == 2010:
        df = df.rename(columns={'R_PL_THEMES': 'RPL_THEMES', 'STATE_NAME': 'STATE'})

    df.FIPS = df.FIPS.astype(int).astype(str).str.zfill(5)
    df['STATE_FIPS'] = df.FIPS.apply(lambda x: x[:2])
    df['CZ_FIPS'] = df.FIPS.apply(lambda x: x[2:])
    df['year'] = yr
    df_list_svi.append(df)

df_svi = pd.concat(df_list_svi)
df_svi = df_svi.reset_index()

# get variables with mostly complete data
x = (df_svi.isnull().sum() * 100 / len(df_svi)).reset_index()
complete_cols = x.loc[x[0] < 25]['index'].to_list()

# interpolate annually
years = np.arange(2000, 2023)
full_years = pd.DataFrame({'year': np.tile(years, len(df_svi.FIPS.unique())), 'FIPS': df_svi.FIPS.unique().repeat(len(years))})
df_svi = full_years.merge(df_svi, on=['year', 'FIPS'], how='left')

df_svi_interp = df_svi[['FIPS'] + complete_cols[5:]].groupby('FIPS').apply(lambda group: group.interpolate(method='index'))
df_svi_interp = df_svi_interp.reset_index(drop=True)

2000
2010
2014
2016
2018
2020
2022


C:\Users\sm6432\AppData\Local\Temp\ipykernel_14912\3000931533.py:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_svi_interp = df_svi[['FIPS'] + complete_cols[5:]].groupby('FIPS').apply(lambda group: group.interpolate(method='index'))
C:\Users\sm6432\AppData\Local\Temp\ipykernel_14912\3000931533.py:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_svi_interp = df_svi[['FIPS'] + complete_cols[5:]].groupby('FIPS').apply(lambda group: group.interpolate(method='index'))
C:\Users\sm6432\AppData\Local\Temp\ipykernel_14912\3000931533.py:38: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_svi_interp = df_svi[['FIPS

In [14]:
# Geography is not available in 2000. Available years include (2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009)
avail_yrs = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009]

census_yr_list = []
for yr in avail_yrs:
    df = download_census_data_for_all_states(census_api_key='fe03ce9cf90615687ec84cd4ab447c787ca39dcf', year=yr)
    df['year'] = yr
    census_yr_list.append(df)

census_data = pd.concat(census_yr_list)

In [22]:
# only keep CONUS
idx = noaa_data['STATE_FIPS'] <= 56.5
noaa_data = noaa_data[idx].reset_index(drop=True)
noaa_data = noaa_data[noaa_data['CZ_TYPE'] != 'M'].reset_index(drop=True) # Marine regions

# format FIPS codes
noaa_data['STATE_FIPS'] = noaa_data['STATE_FIPS'].astype(int).astype(str).str.zfill(2)
noaa_data['CZ_FIPS'] = noaa_data['CZ_FIPS'].astype(int).astype(str).str.zfill(3)
fips_to_abbreviation_map = {s.fips: s.abbr for s in states.STATES}
noaa_data['STATE_ABBREVIATION'] = noaa_data['STATE_FIPS'].map(fips_to_abbreviation_map)
noaa_data['STATE_ZONE'] = noaa_data['STATE_ABBREVIATION'] + noaa_data['CZ_FIPS']


county_zone_mapping_trimmed = county_zone_mapping[['STATE_ZONE', 'FIPS', 'LAT', 'LON']].copy()
noaa_data = noaa_data.merge(
    county_zone_mapping_trimmed,
    on='STATE_ZONE',
    how='left'
)

# Now format the COUNTY_FIPS column
noaa_data['COUNTY_FIPS'] = np.nan
noaa_data['COUNTY_FIPS'] = noaa_data['COUNTY_FIPS'].astype(str)
noaa_data.loc[noaa_data['CZ_TYPE'] == 'C', 'COUNTY_FIPS'] = noaa_data.loc[noaa_data['CZ_TYPE'] == 'C', 'CZ_FIPS']
noaa_data.loc[noaa_data['CZ_TYPE'] == 'Z', 'COUNTY_FIPS'] = noaa_data.loc[noaa_data['CZ_TYPE'] == 'Z', 'FIPS'].str[2:]
noaa_data['STATE_COUNTY_FIPS'] = noaa_data['STATE_FIPS'] + noaa_data['COUNTY_FIPS']

# Format Damage Columns
noaa_data['DAMAGE_PROPERTY'] = noaa_data['DAMAGE_PROPERTY'].apply(decode_damage_property)
noaa_data['DAMAGE_CROPS'] = noaa_data['DAMAGE_CROPS'].apply(decode_damage_property)

In [23]:
noaa_data.EVENT_TYPE.unique()

array(['Tornado', 'Hail', 'Thunderstorm Wind',
       'TORNADOES, TSTM WIND, HAIL', 'THUNDERSTORM WINDS/FLOODING',
       'HAIL/ICY ROADS', 'HAIL FLOODING',
       'THUNDERSTORM WINDS/FLASH FLOOD', 'THUNDERSTORM WINDS LIGHTNING',
       'THUNDERSTORM WIND/ TREES', 'THUNDERSTORM WIND/ TREE',
       'THUNDERSTORM WINDS FUNNEL CLOU', 'TORNADO/WATERSPOUT',
       'THUNDERSTORM WINDS/HEAVY RAIN', 'THUNDERSTORM WINDS HEAVY RAIN',
       'THUNDERSTORM WINDS/ FLOOD', 'High Wind', 'Flash Flood',
       'Winter Storm', 'Blizzard', 'Cold/Wind Chill', 'Heavy Snow',
       'Flood', 'Ice Storm', 'Dense Fog', 'Winter Weather', 'Avalanche',
       'Frost/Freeze', 'Lightning', 'Heat', 'Heavy Rain', 'Funnel Cloud',
       'Coastal Flood', 'Strong Wind', 'Wildfire', 'Waterspout',
       'High Surf', 'Dust Storm', 'Drought', 'Rip Current', 'Dust Devil',
       'Tropical Storm', 'Debris Flow', 'Hurricane (Typhoon)',
       'Freezing Fog', 'Storm Surge/Tide', 'Marine High Wind', 'Sleet',
       'Lake-Effect

In [24]:
# only keep hurricane events
# hurricane_event_names = ['Hurricane', 'Tropical Depression', 'Hurricane (Typhoon)', 'Tropical Storm']
hurricane_event_names = ['Flood', 'Flash Flood', 'Heavy Rain']
noaa_data = noaa_data[noaa_data['EVENT_TYPE'].isin(hurricane_event_names)].reset_index(drop=True)

# drop hurricane events that caused 0 or very low total damage anywhere in the US (likely just an ocean event, check this)
# storms_to_drop = ['200208', '200705', '200706', '200708', '200710', '200711','201209', '201706', '201805',
#  '202006', '202106', '202206', '202207', '202411']
idx = noaa_data['BEGIN_YEARMONTH'].astype(str).isin(storms_to_drop)

noaa_data = noaa_data[~noaa_data['BEGIN_YEARMONTH'].astype(str).isin(storms_to_drop)].reset_index(drop=True)

damages = noaa_data[['STATE_COUNTY_FIPS', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'BEGIN_YEARMONTH', 'LAT', 'LON']].copy()
damages = damages.dropna(subset=['STATE_COUNTY_FIPS']).reset_index(drop=True)
damages['year'] = damages.BEGIN_YEARMONTH.astype(str).map(lambda x: int(x[:4]))

In [15]:
census_data['state'] = census_data['state'].astype(int).astype(str).str.zfill(2)
census_data['county'] = census_data['county'].astype(int).astype(str).str.zfill(3)
census_data['tract'] = census_data['tract'].astype(int).astype(str).str.zfill(6)

census_data['STATE_COUNTY_FIPS'] = census_data['state'] + census_data['county']
census_data['GEOID'] = census_data['state'] + census_data['county'] + census_data['tract']

census_data.loc[census_data['B19013_001E'] < -1., 'B19013_001E'] = np.nan

In [16]:
# join census api data with svi census data
census_data = census_data.merge(df_svi_interp, left_on=['STATE_COUNTY_FIPS', 'year'], right_on=['FIPS', 'year'], how='left')

In [25]:
tract_data = census_data.merge(
    damages,
    on=['STATE_COUNTY_FIPS', 'year'],
    how='inner'
)

tract_data['LAT'] = tract_data['LAT'].astype(np.float64)
tract_data['LON'] = tract_data['LON'].astype(np.float64)

tract_data = tract_data[tract_data['BEGIN_YEARMONTH'] > 200000].reset_index(drop=True)

states_to_drop = []
number_of_data_points_ = []
for state in tract_data['state'].unique():
    idx = tract_data['state'] == state
    number_of_data_points = tract_data[idx].shape[0]

    if number_of_data_points < 1_000:
        # print(number_of_data_points)
        states_to_drop.append(state)
    number_of_data_points_.append(number_of_data_points)



tract_data = tract_data[~tract_data['state'].isin(states_to_drop)].reset_index(drop=True)


In [26]:
# Clean data
# df_clean = tract_data[
#     ['B01001_001E', 'B19001_001E', 'B19013_001E', 'DAMAGE_PROPERTY', 'county', 'state', 'BEGIN_YEARMONTH',
#      'B19001_002E', 'B19001_003E', 'B19001_004E', 'B19001_005E', 'B19001_006E',
#     'B19001_007E', 'B19001_008E', 'B19001_009E', 'B19001_010E', 'B19001_011E',
#     'B19001_012E', 'B19001_013E', 'B19001_014E', 'B19001_015E', 'B19001_016E', 'B19001_017E'
#      ]].dropna()

df_clean = tract_data

# # Inputs (log-scale demographics, raw damages)
# pop = torch.tensor(np.log1p(df_clean['B01001_001E'].values), dtype=DTYPE, device=DEVICE)
# households = torch.tensor(np.log1p(df_clean['B19001_001E'].values), dtype=DTYPE, device=DEVICE)
# income = torch.tensor(np.log1p(df_clean['B19013_001E'].values), dtype=DTYPE, device=DEVICE)
# damage_property = torch.tensor(df_clean['DAMAGE_PROPERTY'].values, dtype=DTYPE, device=DEVICE)

# Define storm_county key
df_clean['storm_county'] = df_clean['BEGIN_YEARMONTH'].astype(str) + '_' + df_clean['state'] + df_clean['county']
# storm_county_idx, storm_county_keys = pd.factorize(df_clean['storm_county'])
# storm_county_idx = torch.tensor(storm_county_idx, dtype=torch.long, device=DEVICE)

# # County-level target (sum of tract damages per county/storm)
# # df_county = df_clean.groupby('storm_county')['DAMAGE_PROPERTY'].sum().reindex(storm_county_keys)
# aggregated_damages_storm_county = df_clean.groupby('storm_county')['DAMAGE_PROPERTY'].sum().reset_index()
# y_county = torch.tensor(aggregated_damages_storm_county['DAMAGE_PROPERTY'].values, dtype=DTYPE, device=DEVICE)

# # income buckets
# income_proportions = df_clean[income_bins].div(df_clean['B19001_001E'], axis=0).fillna(0)
# income_group_tensor = torch.tensor(income_proportions.values, dtype=DTYPE, device=DEVICE)
# # Sizes
# num_tracts = len(df_clean)
# num_counties = len(storm_county_keys)



In [28]:
yrs = df_clean.year.unique()

for yr in yrs:
    if yr != 2023: # no svi data for 2023
        df_yr = df_clean.loc[df_clean.year == yr]
        # df_yr.to_csv('data/df_clean/df_hurricane_' + str(yr) + '.csv', index=False)
        df_yr.to_csv('data/df_clean/df_flood_' + str(yr) + '.csv', index=False)